In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from bnb.problem import OptimizationProblem
from matplotlib import pyplot as plt
import matplotlib
from scipy.optimize import minimize
import pandas as pd
from bnb.fml_solver import FMLSolver
import warnings
import time
import seaborn as sns
sns.set()
matplotlib.rcParams.update({'font.size': 11, 'font.family': 'serif'})

In [5]:
np.random.seed(1)
n, m = 10, 3

a0 = np.random.uniform(0, 4, size=(m, n))
b0 = np.random.uniform(0.001, 0.01, size=n)
w0 = np.random.uniform(size=m)
w0 /= np.sum(w0)

problem = OptimizationProblem(a0, b0, w0)



In [25]:
z_lb = np.exp(-problem.p_ub * problem.b)
z_ub = np.exp(-problem.p_lb * problem.b)
S = np.exp(problem.A)

def cnstr_1(theta, c):
    z = theta[m:]
    return 1 / problem.x_lb[c] - 1 - S[c] @ z  # >= 0

def cnstr_2(theta, c):
    z = theta[m:]
    return 1 + S[c] @ z - 1 / problem.x_ub[c]   # >= 0

constraints = (
    [{"type": "ineq", "fun": lambda theta, c=c: cnstr_1(theta, c)} for c in range(m)] +
    [{"type": "ineq", "fun": lambda theta, c=c: cnstr_2(theta, c)} for c in range(m)]
)

theta_start = np.hstack(((problem.x_ub + problem.x_lb) / 2, (z_ub + z_lb) / 2))

bounds = list(zip(problem.x_lb, problem.x_ub)) + list(zip(z_lb, z_ub))

def objective(theta):
    x, z = theta[:m], theta[m:]
    return np.sum(z * np.log(z) * (problem.k @ x))

opt = minimize(
    objective,
    theta_start,
    bounds=bounds,
    constraints=constraints
)

opt

     fun: -3694.4727843837227
     jac: array([-2.50951657e+03, -2.68474533e+03, -2.37769562e+03,  6.10351562e-05,
        6.10351562e-05,  0.00000000e+00,  6.10351562e-05,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.5226932 , 0.59345371, 0.33204144, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944])

In [6]:
# from scipy.optimize import LinearConstraint

# constr_1 = LinearConstraint(S, -np.inf, 1 / problem.x_lb - 1)
# constr_2 = LinearConstraint(-S, -np.inf,  1 - 1 / problem.x_ub)

# constraints = [constr_1, constr_2]



TypeError: Constraints must be defined using a dictionary.

In [59]:
cnstr_2(theta_start)

array([ -8.80413801, -16.39376441, -35.66130005])